#Analyze source content
Using standardized NLP functions and methods. Based on sota library (Langchain) and llm (Openai).

###Load dependencies

In [ ]:
!pip install --upgrade --force-reinstall Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.5.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.4 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0


In [ ]:
!pip install langchain openai tiktoken chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Dependencies document loading Directoryloader needs Pillow 9.5.0
!pip install unstructured[local-inference]
!pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2"
!pip install layoutparser[layoutmodels,tesseract]
!apt-get update
!apt-get install -y libmagic-dev poppler-utils tesseract-ocr
!pip install --ignore-installed Pillow==9.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 84.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 84.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.6) to /tmp/pip-install-650ob2aj/detectron2_ae18362849824a9296d7851da9f2e80e
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-650ob2aj/detectron2_ae18362849824a9296d7851da9f2e80e
  Running command git checkout -q d1e04565d3bec8719335b88be9e9b961bf3ec464
  Resolved https://github.com/facebookresearch/detectron2.git to commit d1e04565d3bec8719335b88be9e9b961bf3ec464
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 14.0 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp31

In [ ]:
# Load environment variables
import os
from pathlib import Path

# Make the display a bit wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# To split our transcript into pieces and search the content effectively
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings

# Our chat model and other functionality to chat with documents, default model is gpt-3.5-turbo
from langchain.llms import OpenAI
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import (
    RetrievalQA,
    RetrievalQAWithSourcesChain
)
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# To create our chat messages
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

###Load credentials

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving credentials.json to credentials (1).json


In [ ]:
#Loading the credentials file
import os
import json
with open('credentials.json', 'r', encoding = 'utf-8') as f:
    credentials = json.load(f)
os.environ['OPENAI_API_KEY'] = credentials['openai_api_key']
openai_api_key = credentials['openai_api_key']

###Setup printing

In [ ]:
#Function for printing output to console and files
#You can call this function at the start of the NLP functions using logger = setup_logging('file_name')

import logging

def setup_logging(function_name):
    # Set up logging
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    # If logger already has handlers, clear them
    if logger.hasHandlers():
        logger.handlers.clear()

    # Create handlers
    c_handler = logging.StreamHandler()
    f_handler = logging.FileHandler(f'{function_name}_log.txt')
    c_handler.setLevel(logging.INFO)
    f_handler.setLevel(logging.INFO)

    # Create formatters and add it to handlers
    format = logging.Formatter('%(message)s')
    c_handler.setFormatter(format)
    f_handler.setFormatter(format)

    # Add handlers to the logger
    logger.addHandler(c_handler)
    logger.addHandler(f_handler)

    return logger

###Load and preprocess data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Load content to vectorstore and setup retriever
gdrive_path = "/content/drive/MyDrive/Data/q_a/moduchain/"
loader = DirectoryLoader(gdrive_path, glob='**/*')
raw_documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=250)
texts = text_splitter.split_documents(raw_documents)

In [ ]:
index = VectorstoreIndexCreator().from_documents(texts)
#index.save_to_disk("knowledgebase.json") 'VectorStoreIndexWrapper' object has no attribute 'save_to_disk'
vector_store = index.vectorstore
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [ ]:
retriever

VectorStoreRetriever(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7f2aac847190>, search_type='similarity', search_kwargs={'k': 1})

In [ ]:
#Preview
print (f'You have {len(raw_documents)} document(s) in your directory')
print (f'There are {len(raw_documents[0].page_content)} characters in your documents')
print (f'After splitting you have {len(texts)} documents')
print ("Preview:")
print (texts[0].page_content, "\n")
print (texts[1].page_content)

You have 2 document(s) in your directory
There are 0 characters in your documents
After splitting you have 32 documents
Preview:
Alles wat jullie zeggen is nieuw voor mij, laat ik het zo zeggen. Misschien kunnen we even beginnen met, want ongetwijfeld, Novolo kennen jullie al een beetje natuurlijk, maar misschien kunnen jullie iets over jullie bedrijf en over julliezelf vertellen? Ja, zal ik even persoonlijk introduceren? We hebben ook een aantal slides die we ook gebruiken als we bij klanten of potentiële klanten zijn. Dus ik denk dat die wel een beetje gaan helpen ook voor jou met content. Persoonlijk, ik ben dus Pepiche, ik ben 32 jaar, ik woon in Breda en heb een dochtertje van 2,5. Ja, zeker leuk, absoluut. Ik ben oorspronkelijk opgeleid in hotelmanagement. Vanuit daar wist ik eigenlijk al vrij snel dat ik niet de hotellerie in ging. Ik had het vrij internationaal georiënteerd geweest. Na de hbo-studie ervoor gekozen om een tussenjaar te doen bij een logistieke dienstverlener in d

#NLP functions

###Search input documents only

In [ ]:
#Search on bare docs itself (availability of vectorstore presumed)
def ask_questions(vectordb, questions):
    logger = setup_logging('ask_questions')
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})
    answers = []
    for question in questions:
        relevant_documents = retriever.get_relevant_documents(question)
        answers.append(f"Question: {question}\n")
        answers.append("Relevant Documents:\n")
        answers.extend(f"{i + 1}. {doc.page_content}\n" for i, doc in enumerate(relevant_documents))
        answers.append("=" * 40 + "\n\n\n")
    for answer in answers:
        logger.info(answer)
    return ''.join(answers)

In [ ]:
#Calling function
questions = [
    "Waarom gaan Moduchain en Novulo een partnership aan?",
    "Kun je uitleggen hoe het partnership implementaties en development beslaat?",
    "Wat voor innovaties zullen er in het warehouse komen?",
    "Wat is citizen development en wat kun je ermee bereiken vindt Moduchain?",
    "Waarom is application composition platform van Novulo te mooi om waar te zijn?",
    # add more questions here
]

# Call the function
answers = ask_questions(retriever, questions)

# If you want to print the returned answers string
print(answers)

Question: Waarom gaan Moduchain en Novulo een partnership aan?

INFO:__main__:Question: Waarom gaan Moduchain en Novulo een partnership aan?

Relevant Documents:

INFO:__main__:Relevant Documents:

1. De komende tijd de combinatie Novolo voor de logistiek. Dus als ik daar het taal naar de componenten zijn dat WMS componenten. Ja, in eerste instantie. Dus we hebben hier een inleiding voor. Ik denk wat belangrijk is waar wij voor staan. En waarom we zo'n goede match hebben met Novolo. Modulair supply chain. Wij staan voor modulairiteit. Dus gebruik wat je nodig hebt. En ook niet teveel, niet een overkeel. Maar je wil ook weer hebben wat je nodig hebt. En plug and play. Wat je ook wel veel ziet bij innovaties in de markt. Dat je heel veel tijd kwijt bent aan integratie ervan. Wij willen steeds meer naar plug and play. Dus makkelijker nieuwe functionaliteiten toevoegen. Daar sluit Novolo natuurlijk perfect op aan. Het moet innovatief zijn. Of innovaties kunnen faciliteren. En wat wij heel 

Question: Waarom gaan Moduchain en Novulo een partnership aan?
Relevant Documents:
1. De komende tijd de combinatie Novolo voor de logistiek. Dus als ik daar het taal naar de componenten zijn dat WMS componenten. Ja, in eerste instantie. Dus we hebben hier een inleiding voor. Ik denk wat belangrijk is waar wij voor staan. En waarom we zo'n goede match hebben met Novolo. Modulair supply chain. Wij staan voor modulairiteit. Dus gebruik wat je nodig hebt. En ook niet teveel, niet een overkeel. Maar je wil ook weer hebben wat je nodig hebt. En plug and play. Wat je ook wel veel ziet bij innovaties in de markt. Dat je heel veel tijd kwijt bent aan integratie ervan. Wij willen steeds meer naar plug and play. Dus makkelijker nieuwe functionaliteiten toevoegen. Daar sluit Novolo natuurlijk perfect op aan. Het moet innovatief zijn. Of innovaties kunnen faciliteren. En wat wij heel erg extra doen is menselijk aanpak. Ja, wat we in de afgelopen jaren wel vaak hebben gezien. Is dat als er een tech

###Search with more context

In [ ]:
#Ask questions using context optimally
def ask_questions_in_context(retriever, questions, context):
  logger = setup_logging('ask_questions_in_context')
  retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

  template = """
    Ik wil dat je als een assistent die onderzoekt acteert. Je geeft zo volledig mogelijk antwoord op mijn vragen.

    Context: {context}
    Question: {question}?
    """

  prompt = PromptTemplate(
        input_variables=["question", "context"],
        template=template,
    )

  llm = OpenAI(temperature=0)
  compressor = LLMChainExtractor.from_llm(llm)
  compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

  results = {}  # Initialize the dictionary here

  for question in questions:
    compressed_docs = compression_retriever.get_relevant_documents(question)
    llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
    llm_chain_output = llm_chain.predict(question=question, context=compressed_docs)
    results[question] = llm_chain_output

  for question, answer in results.items():
        logger.info(f'Question: {question}\nAnswer: {answer}\n')

  return results

In [ ]:
#Calling function
questions = [
    "Waarom gaan Moduchain en Novulo een partnership aan?",
    "Kun je uitleggen hoe het partnership implementaties en development beslaat?",
    "Wat voor innovaties zullen er in het warehouse komen?",
    "Wat is citizen development en wat kun je ermee bereiken vindt Moduchain?",
    "Waarom is application composition platform van Novulo te mooi om waar te zijn?",
]

context = "Moduchain gelooft in modulaire supply chains, waarbij de inzet van het Novulo application platform ervoor zorgt dat er gemakkelijk functionaliteiten zijn toe te voegen."  # replace with actual context
answers = ask_questions_in_context(retriever, questions, context)

ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='app.posthog.com', port=443): Read timed out. (read timeout=15))




> Entering new LLMChain chain...
Prompt after formatting:

    Ik wil dat je als een assistent die onderzoekt acteert. Je geeft zo volledig mogelijk antwoord op mijn vragen.

    Context: [Document(page_content='"Modulair supply chain. Wij staan voor modulairiteit. Dus gebruik wat je nodig hebt. En ook niet teveel, niet een overkeel. Maar je wil ook weer hebben wat je nodig hebt. En plug and play. Wat je ook wel veel ziet bij innovaties in de markt. Dat je heel veel tijd kwijt bent aan integratie ervan. Wij willen steeds meer naar plug and play. Dus makkelijker nieuwe functionaliteiten toevoegen. Daar sluit Novolo natuurlijk perfect op aan. Het moet innovatief zijn. Of innovaties kunnen faciliteren."', metadata={'source': '/content/drive/MyDrive/Data/q_a/moduchain/transcriptie-moduchain.txt'})]
    Question: Waarom gaan Moduchain en Novulo een partnership aan??
    


INFO:backoff:Backing off send_request(...) for 0.6s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='app.posthog.com', port=443): Read timed out. (read timeout=15))



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    Ik wil dat je als een assistent die onderzoekt acteert. Je geeft zo volledig mogelijk antwoord op mijn vragen.

    Context: []
    Question: Kun je uitleggen hoe het partnership implementaties en development beslaat??
    


INFO:backoff:Backing off send_request(...) for 0.4s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='app.posthog.com', port=443): Read timed out. (read timeout=15))
INFO:backoff:Backing off send_request(...) for 1.1s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='app.posthog.com', port=443): Read timed out. (read timeout=15))


...

ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='app.posthog.com', port=443): Read timed out. (read timeout=15))




> Entering new LLMChain chain...
Prompt after formatting:

    Ik wil dat je als een assistent die onderzoekt acteert. Je geeft zo volledig mogelijk antwoord op mijn vragen.

    Context: [Document(page_content='"Modulair supply chain. Wij staan voor modulairiteit. Dus gebruik wat je nodig hebt. En ook niet teveel, niet een overkeel. Maar je wil ook weer hebben wat je nodig hebt. En plug and play. Wat je ook wel veel ziet bij innovaties in de markt. Dat je heel veel tijd kwijt bent aan integratie ervan. Wij willen steeds meer naar plug and play. Dus makkelijker nieuwe functionaliteiten toevoegen. Daar sluit Novolo natuurlijk perfect op aan. Het moet innovatief zijn. Of innovaties kunnen faciliteren."', metadata={'source': '/content/drive/MyDrive/Data/q_a/moduchain/transcriptie-moduchain.txt'})]
    Question: Wat voor innovaties zullen er in het warehouse komen??
    



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    Ik wil dat je als een assistent die onderzoekt acteert. Je geeft zo volledig mogelijk antwoord op mijn vragen.

    Context: []
    Question: Wat is citizen development en wat kun je ermee bereiken vindt Moduchain??
    

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    Ik wil dat je als een assistent die onderzoekt acteert. Je geeft zo volledig mogelijk antwoord op mijn vragen.

    Context: [Document(page_content='"Het is nog geen jaar dat Gartner er heel stevig mee komt. Ze hebben het bij model IT in gereild voor application composition. Om bedrijven aan het innoveren en te krijgen. En verbindbaar te maken. Ja. We hadden wel sterk het gevoel in het begin toen we hier mee in aanraking kwamen. Dat het een beetje zo\'n to good to be true verhaal is."', metadata={'source': '/content/drive/MyDrive/Data/q_a/moduchain/transcriptie-moduchain.txt'})]
    Question: Waarom is a

Question: Waarom gaan Moduchain en Novulo een partnership aan?
Answer: 
Antwoord: Moduchain en Novolo gaan een partnership aan omdat Novolo perfect aansluit op de modulairiteit die Moduchain voorstaat. Novolo biedt innovatieve functionaliteiten die makkelijk toe te voegen zijn, waardoor Moduchain in staat is om snel nieuwe innovaties te implementeren.

INFO:__main__:Question: Waarom gaan Moduchain en Novulo een partnership aan?
Answer: 
Antwoord: Moduchain en Novolo gaan een partnership aan omdat Novolo perfect aansluit op de modulairiteit die Moduchain voorstaat. Novolo biedt innovatieve functionaliteiten die makkelijk toe te voegen zijn, waardoor Moduchain in staat is om snel nieuwe innovaties te implementeren.

Question: Kun je uitleggen hoe het partnership implementaties en development beslaat?
Answer: 
Answer: Ja, het partnership implementaties en development beslaat een aantal verschillende aspecten. Ten eerste zal het partnership de ontwikkeling van nieuwe producten en diensten 


> Finished chain.


###LLM augmented search

In [ ]:
#Augmenting search with LLM
def ask_questions_augmented(retriever, openai_api_key, queries):
  logger = setup_logging('ask_questions_augmented')
  llm = ChatOpenAI(
        openai_api_key=openai_api_key,
        model_name='gpt-3.5-turbo',
        temperature=0.0
        )

  qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1}),
        verbose=True,
        input_key="query"
        )

  results = {}
  for query in queries:
        result = qa.run(query)
        results[query] = result

  for query, result in results.items():
        logger.info(f'Query: {query}\nAnswer: {result}\n')
        logger.info("=" * 40)

  return results

In [ ]:
#Calling the function
queries = [
    "Waarom gaan Moduchain en Novulo een partnership aan?",
    "Kun je uitleggen hoe het partnership implementaties en development beslaat?",
    "Wat voor innovaties zullen er in het warehouse komen?",
    "Wat is citizen development en wat kun je ermee bereiken vindt Moduchain?",
    "Waarom is application composition platform van Novulo te mooi om waar te zijn?",
    # add more questions here
]

# Set your OpenAI API key
openai_api_key = openai_api_key

# Call the function
answers = ask_questions_augmented(retriever, openai_api_key, queries)

# If you want to print the returned answers dictionary
for query, answer in answers.items():
    print(f"Query: {query}\nAnswer: {answer}\n" + "=" * 40)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...


Query: Waarom gaan Moduchain en Novulo een partnership aan?
Answer: Moduchain en Novulo gaan een partnership aan omdat Moduchain staat voor modulaire supply chain en plug-and-play-functionaliteiten, terwijl Novulo innovatieve oplossingen biedt die gemakkelijk kunnen worden geïntegreerd. Bovendien hecht Moduchain veel waarde aan een menselijke aanpak bij technische implementaties, wat ook goed past bij de werkwijze van Novulo. Door samen te werken kunnen ze hun krachten bundelen en hun klanten beter bedienen.

INFO:__main__:Query: Waarom gaan Moduchain en Novulo een partnership aan?
Answer: Moduchain en Novulo gaan een partnership aan omdat Moduchain staat voor modulaire supply chain en plug-and-play-functionaliteiten, terwijl Novulo innovatieve oplossingen biedt die gemakkelijk kunnen worden geïntegreerd. Bovendien hecht Moduchain veel waarde aan een menselijke aanpak bij technische implementaties, wat ook goed past bij de werkwijze van Novulo. Door samen te werken kunnen ze hun kracht


> Finished chain.
Query: Waarom gaan Moduchain en Novulo een partnership aan?
Answer: Moduchain en Novulo gaan een partnership aan omdat Moduchain staat voor modulaire supply chain en plug-and-play-functionaliteiten, terwijl Novulo innovatieve oplossingen biedt die gemakkelijk kunnen worden geïntegreerd. Bovendien hecht Moduchain veel waarde aan een menselijke aanpak bij technische implementaties, wat ook goed past bij de werkwijze van Novulo. Door samen te werken kunnen ze hun krachten bundelen en hun klanten beter bedienen.
Query: Kun je uitleggen hoe het partnership implementaties en development beslaat?
Answer: Ja, natuurlijk. Het partnership van deze organisatie omvat zowel implementaties als ontwikkeling. Ze bieden implementatietrajecten aan als een solution provider en willen ook componenten ontwikkelen, voornamelijk op het gebied van WMS, TMS en OMS functionaliteiten. Het is dus een combinatie van consultancy en ontwikkeling. Ze vinden het belangrijk om de menselijke aanpak te

###LLM augmented search with sources

In [ ]:
def ask_questions_augmented_with_sources(retriever, vectordb, questions):
    logger = setup_logging('ask_questions_augmented_with_sources')
    chain = RetrievalQAWithSourcesChain.from_chain_type(OpenAI(temperature=0), chain_type="stuff", retriever=vector_store.as_retriever())
    qa_chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
    qa = RetrievalQAWithSourcesChain(combine_documents_chain=qa_chain, retriever=vector_store.as_retriever())

    results = {}
    for question in questions:
        result = chain({"question": question}, {"input_documents": vectordb})
        results[question] = result

    for question, answer in results.items():
        logger.info(f'Question: {question}\nAnswer: {answer}\n')

    return results

In [ ]:
#Calling the function
questions = [
"Waarom gaan Moduchain en Novulo een partnership aan?",
    "Kun je uitleggen hoe het partnership implementaties en development beslaat?",
    "Wat voor innovaties zullen er in het warehouse komen?",
    "Wat is citizen development en wat kun je ermee bereiken vindt Moduchain?",
    "Waarom is application composition platform van Novulo te mooi om waar te zijn?",
    # add more questions here
]

# Call the function
answers = ask_questions_augmented_with_sources(retriever, vector_store, questions)

Question: Waarom gaan Moduchain en Novulo een partnership aan?
Answer: {'answer': ' Moduchain en Novulo gaan een partnership aan om innovatieve oplossingen te implementeren bij bedrijven met supply chain vragenstukken, zoals logistieke dienstverleners, productiebedrijven en maakindustrie.\n', 'sources': '/content/drive/MyDrive/Data/q_a/moduchain/transcriptie-moduchain.txt'}

INFO:__main__:Question: Waarom gaan Moduchain en Novulo een partnership aan?
Answer: {'answer': ' Moduchain en Novulo gaan een partnership aan om innovatieve oplossingen te implementeren bij bedrijven met supply chain vragenstukken, zoals logistieke dienstverleners, productiebedrijven en maakindustrie.\n', 'sources': '/content/drive/MyDrive/Data/q_a/moduchain/transcriptie-moduchain.txt'}

Question: Kun je uitleggen hoe het partnership implementaties en development beslaat?
Answer: {'answer': ' Moduchain biedt een partnership aan waarbij zij het NoFlow platform implementeren en componenten ontwikkelen op het WMS, TM